In [22]:
import env
import acquire
import requests as req
import pandas as pd
import utilities as utils
import re

In [52]:
bootcamp_readme_df.to_csv(env.data_path + "coding_bootcamp_readme.csv", index=False)

In [2]:
bootcamp_readme_df = pd.read_csv(env.data_path + "coding_bootcamp_readme.csv")

In [3]:
bootcamp_readme_df

,repo,language,readme_contents
0,Pierian-Data/Complete-Python-3-Bootcamp,Jupyter Notebook,# Complete-Python-3-Bootcamp\nCourse Files for...
1,microsoft/frontend-bootcamp,TypeScript,# Frontend Bootcamp / Days in the Web\n\n> 🚨🚨 ...
2,vitorfs/bootcamp,CSS,Bootcamp\n========\n\nAn enterprise oriented s...
3,jmportilla/Complete-Python-Bootcamp,Jupyter Notebook,# Complete-Python-Bootcamp\n\n## Welcome to th...
4,rocketseat-education/bootcamp-gostack-desafios,NaN,"<img alt=""GoStack"" src=""https://storage.google..."
...,...,...,...
993,jdcar/bootcamp-links,NaN,# bootcamp-links\nThe pinned resources from NU...
994,badjatya/web-bootcamp,HTML,# web-bootcamp
995,arehmandev/Terraform-Bootcamp,HCL,# DevOps Bootcamp: AWS-Terraform\n\nEach folde...
996,dbdoyle182/Bootcamp-Overflow,HTML,# Bootcamp_Overflow\n\nAn application inspired...


In [4]:
bootcamp_readme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             998 non-null    object
 1   language         842 non-null    object
 2   readme_contents  880 non-null    object
dtypes: object(3)
memory usage: 23.5+ KB


In [5]:
positions = utils.nan_null_empty_check(bootcamp_readme_df)

NaN values
     rows  columns
0       4        1
1      15        1
2      17        2
3      19        1
4      23        1
..    ...      ...
269   983        1
270   987        1
271   989        1
272   990        1
273   993        1

[274 rows x 2 columns]
--------------------------------
Empty values
   rows  columns
0   514        2
1   565        2
2   635        2
--------------------------------


In [8]:
bootcamp_readme_df.iloc[positions['nan_positions'][0]]

,repo,language,readme_contents
4,rocketseat-education/bootcamp-gostack-desafios,NaN,"<img alt=""GoStack"" src=""https://storage.google..."
15,panacloud/bootcamp-2020,NaN,# Bootcamp 2020: Learn to Build Modern Full St...
17,golang-book/bootcamp-examples,Go,NaN
19,glaucia86/frontend-bootcamp-online,NaN,# Front-End Bootcamp Online (Em Desenvolviment...
23,rocketseat-education/bootcamp-gostack-desafio-02,NaN,"<h1 align=""center"">\n <img alt=""Fastfeet"" tit..."
...,...,...,...
983,PacktPublishing/Linux-Administration-Bootcamp-...,NaN,# Linux-Administration-Bootcamp-Go-from-Beginn...
987,1Fungi99/BootCamp,NaN,# BootCamp
989,highkay/bootcamp,NaN,# 前言\n\n这里记录了我平时工作学习中看到的比较深入的和有价值的文章，希望可以和团队里的...
990,JavierParral/Bootcamp,NaN,# Bootcamp Credilikeme\n\nEl bootcamp de credi...


In [9]:
bootcamp_readme_df.iloc[positions['empty_positions'][0]]

,repo,language,readme_contents
514,OpenMined/Bootcamps,NaN,\n
565,MrEvil22/Udemy_Complete_Python_Bootcamp_Go_fro...,Jupyter Notebook,\n
635,momentmuse/modern-js-bootcamp,JavaScript,\n


In [10]:
drop_rows = list(positions['empty_positions'][0]) + list(positions['nan_positions'][0])
drop_rows = list(set(drop_rows))

In [11]:
bootcamp_readme_df = bootcamp_readme_df.drop(index=drop_rows)

In [12]:
bootcamp_readme_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 0 to 997
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             732 non-null    object
 1   language         732 non-null    object
 2   readme_contents  732 non-null    object
dtypes: object(3)
memory usage: 22.9+ KB


In [13]:
bootcamp_readme_df.language.value_counts()

JavaScript                 184
Jupyter Notebook           157
HTML                        99
Python                      73
Java                        37
C#                          24
Shell                       23
CSS                         14
TypeScript                  13
Ruby                         9
PHP                          9
TeX                          8
C                            8
Kotlin                       7
C++                          6
PowerShell                   5
HCL                          5
Scala                        5
Clojure                      5
Objective-C                  4
R                            4
Go                           4
Makefile                     3
Dockerfile                   3
Swift                        2
Groovy                       2
Dart                         2
Processing                   2
SWIG                         1
SCSS                         1
Jinja                        1
Lasso                        1
Cuda    

In [19]:
bootcamp_readme_df.readme_contents = bootcamp_readme_df.readme_contents.apply(utils.nlp_basic_clean)

In [24]:
# Basic clean did not remove new line characters so remove them here.
bootcamp_readme_df.readme_contents = bootcamp_readme_df.readme_contents.apply(lambda readme : re.sub(r"\n", " ", readme))

In [26]:
bootcamp_readme_df.readme_contents = bootcamp_readme_df.readme_contents.apply(utils.nlp_tokenize)

In [28]:
bootcamp_readme_df.readme_contents = bootcamp_readme_df.readme_contents.apply(utils.nlp_remove_stopwords)

In [31]:
bootcamp_readme_df.readme_contents = bootcamp_readme_df.readme_contents.apply(utils.nlp_lemmatize)